### MLP in PyTorch

References:
- https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
- https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [ ]:
import torch
import numpy as np

from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_classification

### Settings

In [ ]:
device = "cpu"

### Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]

        return x, y

In [ ]:
# Dataset config
n_samples = 1000
n_features = 20
n_informative = 12

X, y = make_classification(
    n_samples=n_samples, n_features=n_features, n_informative=n_informative
)

y = np.expand_dims(y, axis=1)

# Cast to float 32
X = X.astype(np.float32)
y = y.astype(np.float32)

### Model

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_fts):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(n_fts, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        logits = self.linear(x)
        return logits

#### Training

In [ ]:
def train_loop(epoch, dataloader, model, loss_fn, optimizer):
    
    # Set train mode
    model.train()

    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Epoch:{epoch} loss: {loss:>7f}  [{current:>5d}/{size:>5d}]", end='\r')   

    # End of epoch
    print(f"Epoch:{epoch} loss: {loss:>7f}  [{size:>5d}/{size:>5d}]") 
    
def test_loop(epoch, dataloader, model, loss_fn, threshold=0.5):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0

    # Set evaluation mode
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            logits = model(X)
            probs = nn.functional.sigmoid(logits)

            val_loss += loss_fn(logits, y).item()
            correct += ((probs > 0.5) == y).type(torch.float).sum().item()

    val_loss /= num_batches
    val_accuracy = correct / size
    print(f"Epoch:{epoch} Val accuracy: {(100*val_accuracy):>0.1f}%, Avg loss: {val_loss:>8f} \n")

In [ ]:
num_epochs = 10

# Model hyperparameters
batch_size = 32
learning_rate = 0.001

# Build DataLoader
train_dataset = CustomDataset(X, y)
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model
model = MLP(n_features).to(device)

# Initialize the loss function
loss_fn = nn.BCEWithLogitsLoss()

# Initalizer loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Train model for `num_epochs
# For simplicity we are evaluating in the same dataset
# You should always evaluate model performance on a separate holdout set
for epoch in range(num_epochs):
    train_loop(epoch, train_dl, model, loss_fn, optimizer)
    test_loop(epoch, train_dl, model, loss_fn)